In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from tensorflow import keras
from keras.layers.core import Dense
from tensorflow.keras.layers import Input , Flatten
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [4]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
img_folder=r'/content/drive/MyDrive/sprints project /Scenes training set'
import splitfolders
splitfolders.ratio(img_folder, output="output",
    seed=1337, ratio=(.7,0.3), group_prefix=None, move=False)

Copying files: 14034 files [05:20, 43.75 files/s] 


In [6]:
train_dir = '/content/output/train'
val_dir = '/content/output/val'

In [7]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen= ImageDataGenerator(rescale=1. /255)
train_generator=train_datagen.flow_from_directory(
     directory=train_dir,
     target_size=(224, 224),
     color_mode= "rgb",
     batch_size=30,
     shuffle= False,
     seed=42 
)

Found 9820 images belonging to 6 classes.


In [8]:
from keras.preprocessing.image import ImageDataGenerator
val_datagen= ImageDataGenerator(rescale=1. /255)
val_generator= val_datagen.flow_from_directory(
     directory=val_dir,
     target_size=(224, 224),
     color_mode= "rgb",
     batch_size=30,
     shuffle= False,
     seed=42 
)

Found 4214 images belonging to 6 classes.


In [9]:
test_dir=r'/content/drive/MyDrive/sprints project /testset'

In [10]:
from keras.preprocessing.image import ImageDataGenerator
test_datagen= ImageDataGenerator(rescale=1. /255)
test_generator= test_datagen.flow_from_directory(
     directory= test_dir,
     target_size=(224, 224),
     color_mode= "rgb",
     batch_size=30,
     shuffle= False,
     seed=42 
)

Found 3050 images belonging to 1 classes.


In [11]:
train_generator.class_indices

{'buildings': 0,
 'forest': 1,
 'glacier': 2,
 'mountain': 3,
 'sea': 4,
 'street': 5}

In [12]:
train_generator.classes

array([0, 0, 0, ..., 5, 5, 5], dtype=int32)

In [13]:
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications.vgg16 import VGG16

In [14]:
IMAGE_SIZE = [224, 224]
vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)

for layer in vgg.layers:
    layer.trainable = False
    
x = Flatten()(vgg.output)


x = Dense(6, activation = 'softmax')(x)
model = Model(inputs = vgg.input, outputs = x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



model.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [15]:
training_images = 9820
validation_images = 4214

history = model.fit(train_generator,
     steps_per_epoch = 306,
     epochs = 30,  
     validation_data= val_generator,
     validation_steps=30

)

Epoch 1/30
306/306 [==============================] - 62s 160ms/step - loss: 1.4523 - accuracy: 0.6657 - val_loss: 0.3859 - val_accuracy: 0.8789
Epoch 2/30
306/306 [==============================] - 50s 161ms/step - loss: 0.7480 - accuracy: 0.8202 - val_loss: 0.3856 - val_accuracy: 0.8978
Epoch 3/30
306/306 [==============================] - 52s 169ms/step - loss: 0.4968 - accuracy: 0.8650 - val_loss: 0.1335 - val_accuracy: 0.9600
Epoch 4/30
306/306 [==============================] - 51s 167ms/step - loss: 0.3431 - accuracy: 0.8922 - val_loss: 0.9739 - val_accuracy: 0.7878
Epoch 5/30
306/306 [==============================] - 52s 170ms/step - loss: 0.4106 - accuracy: 0.8918 - val_loss: 0.4356 - val_accuracy: 0.8989
Epoch 6/30
306/306 [==============================] - 51s 167ms/step - loss: 0.3385 - accuracy: 0.9153 - val_loss: 0.3594 - val_accuracy: 0.9156
Epoch 7/30
306/306 [==============================] - 52s 169ms/step - loss: 0.2208 - accuracy: 0.9316 - val_loss: 0.5937 - val_ac

In [16]:
model.evaluate_generator(generator=train_generator,steps=30)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


[0.000513545994181186, 1.0]

In [17]:
model.evaluate_generator(generator=val_generator,steps=30)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


[0.7904208302497864, 0.9200000166893005]

In [23]:
import os 
import glob
import pandas as pd

Name = []

for img in glob.glob(test_dir):
    prediction = model.predict_generator(test_generator,steps=3050,verbose=1)
    for name in os.listdir('/content/drive/MyDrive/sprints project /testset/Scenes testing test'):
         Name.append(name[0::])



predicted_class_indics=np.argmax(prediction,axis=1)
Name.sort()
Temp = {'Image':Name, 'Label':predicted_class_indics}
temp = pd.DataFrame(Temp)
temp.to_csv('/content/results.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


 102/3050 [>.............................] - ETA: 7:18

3050/3050 [==============================] - 15s 5ms/step


In [ ]:
temp

,Image,Label
0,1020.jpg,0
1,103.jpg,0
2,1053.jpg,0
3,1081.jpg,0
4,1082.jpg,0
...,...,...
3045,920.jpg,0
3046,949.jpg,0
3047,952.jpg,0
3048,961.jpg,0
